# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [2]:
import pandas as pd

In [72]:
df = pd.read_csv('zillow_data.csv')
df.head()
print(df.shape)

(14723, 272)


In [80]:
melted = melt_data(df)
melted.dtypes

RegionID               int64
RegionName             int64
City                  object
State                 object
Metro                 object
CountyName            object
SizeRank               int64
Month         datetime64[ns]
Mean Value           float64
dtype: object

In [95]:
import datetime
modern = melted[(melted.Month >= datetime.datetime(2017, 1, 1))]

In [99]:
modern_grouped = modern.groupby('City').mean()
modern_grouped = modern_grouped.drop(['RegionID', 'RegionName', 'SizeRank'], axis = 1)
modern_grouped

,Mean Value
City,
Abbeville,71456.250000
Aberdeen,133578.750000
Abilene,124636.458333
Abingdon,154618.750000
Abington,302140.000000
...,...
Zionsville,375943.750000
Zionville,171731.250000
Zumbro Falls,252293.750000


In [102]:
modern_grouped.sort_values(by = ['Mean Value'], ascending = False)


,Mean Value
City,
Atherton,6280018.75
Palm Beach,4531050.00
Los Altos,3540318.75
Portola Valley,3432131.25
Newport Coast,3286656.25
...,...
East Spencer,27343.75
Witt,25562.50
Earlington,24356.25


In [130]:
boston = melted[(melted.City == 'Boston') & (melted.State == 'MA')]
len(boston.groupby('RegionName').mean())

18

In [131]:
boston

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,Month,Mean Value
561,58649,2135,Boston,MA,Boston,Suffolk,562,1996-04-01,165200.0
1169,58638,2124,Boston,MA,Boston,Suffolk,1170,1996-04-01,100900.0
1511,58641,2127,Boston,MA,Boston,Suffolk,1512,1996-04-01,124600.0
1670,58644,2130,Boston,MA,Boston,Suffolk,1671,1996-04-01,185900.0
1921,58642,2128,Boston,MA,Boston,Suffolk,1922,1996-04-01,83300.0
...,...,...,...,...,...,...,...,...,...
3891997,58636,2122,Boston,MA,Boston,Suffolk,5126,2018-04-01,530200.0
3892177,58640,2126,Boston,MA,Boston,Suffolk,5306,2018-04-01,419400.0
3894372,58634,2120,Boston,MA,Boston,Suffolk,7501,2018-04-01,706600.0
3895014,58625,2111,Boston,MA,Boston,Suffolk,8143,2018-04-01,593600.0


In [134]:
#boston.drop(['RegionID','City','State','Metro','CountyName','SizeRank'], axis=1, inplace=True)
#boston.reset_index(drop=True, inplace=True)
boston

,RegionName,Month,Mean Value
0,2135,1996-04-01,165200.0
1,2124,1996-04-01,100900.0
2,2127,1996-04-01,124600.0
3,2130,1996-04-01,185900.0
4,2128,1996-04-01,83300.0
...,...,...,...
4546,2122,2018-04-01,530200.0
4547,2126,2018-04-01,419400.0
4548,2120,2018-04-01,706600.0
4549,2111,2018-04-01,593600.0


In [136]:
boston['RegionName'].value_counts()

2134    265
2130    265
2125    265
2120    265
2124    265
2128    265
2132    265
2136    265
2111    265
2119    265
2127    265
2131    265
2135    265
2110    265
2122    265
2126    265
2121    265
2116     46
Name: RegionName, dtype: int64

In [138]:
boston = boston[~boston.RegionName.isin([2116])]
boston.head()

,RegionName,Month,Mean Value
0,2135,1996-04-01,165200.0
1,2124,1996-04-01,100900.0
2,2127,1996-04-01,124600.0
3,2130,1996-04-01,185900.0
4,2128,1996-04-01,83300.0


In [139]:
boston['RegionName'].value_counts()

2134    265
2111    265
2125    265
2120    265
2124    265
2128    265
2132    265
2136    265
2119    265
2130    265
2127    265
2131    265
2135    265
2110    265
2122    265
2126    265
2121    265
Name: RegionName, dtype: int64

# Step 2: Data Preprocessing

In [ ]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

# Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

# Step 4: Reshape from Wide to Long Format

In [70]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'], var_name='Month', value_name='Mean Value')
    melted['Month'] = pd.to_datetime(melted['Month'], infer_datetime_format=True)
    melted = melted.dropna(subset=['Mean Value'])
    return melted

# Step 5: ARIMA Modeling

# Step 6: Interpreting Results